In [ ]:
import os
import json
import pandas as pd
import traceback
from langchain import PromptTemplate,HuggingFaceHub,LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
KEY=os.getenv("GOOGLE_API_KEY")
print('KEY')

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,temperature=0.5)  
llm
